In [22]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from numpy import linalg as LA
# Set default font size for plots:
font = {'size'   : 20}
plt.rc('font',**font)

#%matplotlib notebook

def mohr_circle_pf(Pf,angle,phi,C):
    '''Function to calculate effect of pore fluids.
    It uses the function mohr_circle(sigma1,sigma3,theta) as a base'''
    #Define the Principal Stresses
    theta=np.linspace(0,180,180) #angles to compute the circle
    [sigma_n,tau_s,sm]=mohr_circle(sigma1,sigma3,theta) #Without Pf
    [sigma_n2,tau_s2,sm2]=mohr_circle(sigma1-Pf,sigma3-Pf,theta) #With Pf
    #angle=30 #The specific angle you want to compute
    [sn,ts,sm]=mohr_circle(sigma1,sigma3,angle)
    [sn2,ts2,sm2]=mohr_circle(sigma1-Pf,sigma3-Pf,angle)

    #Plot Mohr's Circle
    plt.figure('Mohr Circle',figsize=(10,10))
    plt.plot(sigma_n,tau_s,'b')
    plt.scatter(sm,0,color='r')
    plt.xlabel('$\sigma_n$'),plt.ylabel(r'$ \tau_s$')
    plt.text(sm*1.01,-0.05*np.max(tau_s),'$\sigma_m$')
    plt.text(sigma1*1.01,-0.05*np.max(tau_s),'$\sigma_1$')
    plt.text(sigma3*0.93,-0.05*np.max(tau_s),'$\sigma_3$')
    plt.scatter(sn,ts,color='r')
    plt.plot([sm,sn],[0,ts],'r')
    plt.axis('equal')
    #Plot Mohr's Circle with Pf
    plt.plot(sigma_n2,tau_s2,'b-.')
    plt.scatter(sm-Pf,0,color='r')
    plt.scatter(sn2,ts2,color='r')
    plt.plot([sm-Pf,sn2],[0,ts2],'r-.')
    plt.axis('equal')
    plt.grid(True)
    #plt.show()
    #Plot the Mohr-Coulomb criterion
    sigma,tau=mohr_coulomb(phi,C,sigma1-Pf)
    plt.plot(sigma,tau,'g')
    plt.show()
    return 

def mohr_circle(sigma1,sigma3,theta):
    '''Calculate Mohr's Circle based on the sigma1 and sigma3 principal stresses
    the angle theta is expressed in degrees.
    sigma_n,tau_s, and sigma_m are the normal, tangential, and mean stresses respectively
    Usage= [sigma_n,tau_s,sm]=mohr_circle(sigma1,sigma3,theta)'''
    theta=theta*np.pi/180
    sigma_m=(sigma1+sigma3)/2
    sigma_n=sigma_m+np.cos(2*theta)*(sigma1-sigma3)/2
    tau_s=np.sin(2*theta)*(sigma1-sigma3)/2     
    return sigma_n,tau_s,sigma_m

def mohr_coulomb(phi,C,sigma1):
    '''Mohr-Coulomb criterion
    phi=internal friction angle,   C='''
    sigma=np.linspace(0,sigma1,20)
    tau=C+np.tan(phi*np.pi/180)*sigma
    #plt.plot(sigma,tau,'g')
    #plt.show()
    return sigma,tau

def griffith(T,sigma1):
    '''The Griffith failure criterion. phi as in Couloumb and T is tensile strength'''
    sigma=np.linspace(2*T,sigma1,100)
    tau=np.sqrt((2*T)**2-4*T*sigma)
    return sigma,tau

def mohr_circle_interact(diffP,Pf,C,mu,h,tectonics):
    #Input parameters
    rho=2670
    rhow=1000
    g=9.8
    sig_v=rho*g*h
    sig_h=rhow*g*h
    C=C*1e6
    #T=T*1e6
    diffP=diffP*1e6
    theta2=(90+np.arctan(mu)*180/np.pi)#The 2*theta angle. theta is the optimal angle of shear faulting
    #Tectonic regimes conditional:
    if tectonics=='reverse':
        sigma3=sig_v
        sigma1=diffP+sigma3
    elif tectonics=='normal':
        sigma1=sig_v
        sigma3=sigma1-diffP#[105]
    elif tectonics=='strike-slip':
        sigma3=sig_v-diffP/2
        sigma1=sig_v+diffP/2
    else:
        print('Only three options: normal, reverse, strike-slip. Please type it correctly!')
    sigma3=sigma3*1e-6
    sigma1=sigma1*1e-6
    #Do the plotting
    theta=np.linspace(0,90,180) #angles to compute the circle
    sigma_n,tau_s,sigma_m=mohr_circle(sigma1,sigma3,theta)
    [sn,ts,sm]=mohr_circle(sigma1,sigma3,0.5*theta2)
    [sigma_n2,tau_s2,sm2]=mohr_circle(sigma1-Pf,sigma3-Pf,theta) #With Pf
    [sn2,ts2,sm2]=mohr_circle(sigma1-Pf,sigma3-Pf,0.5*theta2)

    #Plot Mohr's Circle
    plt.figure('Mohr Circle',figsize=(12,10))
    plt.title('$\sigma_1=$'+np.str(np.round(sigma1,decimals=2))+' MPa, $\sigma_3=$'+np.str(np.round(sigma3,decimals=2))+' MPa')
    plt.plot(sigma_n,tau_s,'b')
    plt.scatter(sm,0,color='r')
    plt.xlabel('$\sigma_n$ (MPa)'),plt.ylabel(r'$ \tau_s$ (MPa)')
    plt.text(sm,-0.05*np.max(tau_s),'$\sigma_m$')
    plt.text(sigma1,-0.05*np.max(tau_s),'$\sigma_1$')
    plt.text(sigma3,-0.05*np.max(tau_s),'$\sigma_3$')
    plt.scatter(sn,ts,color='r')
    plt.plot([sm,sn],[0,ts],'r')
    plt.grid(True)
    plt.axis('equal')
    #Plot Mohr's Circle with Pf
    plt.plot(sigma_n2,tau_s2,'b-.')
    plt.scatter(sm-Pf,0,color='r')
    plt.scatter(sn2,ts2,color='r')
    plt.plot([sm-Pf,sn2],[0,ts2],'r-.')
    plt.axis('equal')
    plt.grid(True)
    #Mohr-Coulomb criterion 
    phi=np.arctan(mu)*180/np.pi
    sigma,tau=mohr_coulomb(phi,C*1e-6,sigma1-Pf)
    plt.plot(sigma,tau,'g')
    plt.show()     
    return

# Mohr's circle (2D) and the Mohr-Coulomb Failure Criterion

The normal and tangential stresses to a plane are expressed as parametric equations:

$$\begin{equation}
\sigma_n=\frac{\sigma_{11}+\sigma_{33}}{2}+\frac{\sigma_{11}-\sigma_{33}}{2}cos(2\theta)\\
\tau_s=\frac{\sigma_{11}-\sigma_{33}}{2}sin(2\theta)
\end{equation}$$

with $\sigma_n$ on the *x-axis* and $\tau_s$ on the *y-axis*.  The angle can be thought of as measured from the normal or vertical. 

## Mohr-Couloumb Failure Criterion

this linear equation describes under which conditions an isotropic material will fail.

$$\tau_s = C + tan(\phi)\sigma_n$$
$$\tau_s = C + \mu\sigma_n$$

with $C$ being the cohesive strength and $\mu_i$ being the coefficient of internal friction.  When the Mohr circle intersects this criterion, you can determine the most likely planes of failure with respect to the principal stresses.

## The Effect of Pore Fluid Pressures

In the upper crust there is not only a pressure related to the overburden, but there's also a pressure exerted by the weight of fluids able to move freely within the pore space of rocks.  If this pore space is interconnected, the outward pressure of the fluids is the same as the *hydrostatic pressure*. The net effect of the pore fluid pressure is to lower the lithostatic stresses, and the resulting pressures are referred to as ***effective pressures***. 

In the interactive figure below, the Mohr Circle as a function of effective stress is depicted with dots and line segments.  This effective stress helps reduce the strength of the rock, therefore making it more prone to fractures.

### Tectonic regimes

The sliders help vary the constants of the medium with the differential pressure ($diffP$) going from $0$ to $300~MPa$. The material properties $C$ and $T$ are in units of $MPa$, while the depth ($h$) is in meters.  The following conditions are shown: 

### Normal failure conditions

The maximum stress ($\sigma_1$) is equal to the vertical stress ($\sigma_v$).

$$\sigma_1=\sigma_v$$
$$\sigma_3=\sigma_1-diffP $$

### Reverse failure conditions

The minimum stress ($\sigma_3$) is equal to the vertical stress. 

$$\sigma_3=\sigma_v$$
$$\sigma_1=\sigma_3+diffP $$

### Strike-slip failure conditions

The strike slip regime in the plots shown assumes that $\sigma_2 \approx (\sigma_1+\sigma_3)/2$.

$$\sigma_1=\sigma_v+diffP/2 $$
$$\sigma_3=\sigma_v-diffP/2 $$

In [24]:
#mohr_circle_interact(diffP,Pf,C,T,mu,h,tec_reg)
interactive(mohr_circle_interact,diffP=(0,300,10),Pf = (0,100,1),C=(0,50,1),mu=(0,0.9,0.15),h=(1000,40000,500),tectonics=['normal','reverse','strike-slip'],continuous_update=True)

interactive(children=(IntSlider(value=150, description='diffP', max=300, step=10), IntSlider(value=50, descrip…